In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import quandl
import scipy.optimize as sco
plt.style.use('fivethirtyeight')
np.random.seed(777)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
!pip install yfinance

In [3]:
import yfinance as yf

# Create a Portfolio
symbols = [
    'AAPL',
    'MSFT',
    'META',
    'AMZN',
    'XOM',
    'JNJ',
    'DIS',
    'T',
    'UPS',
    'HSY'
]

# Get the stock data
data = yf.download(symbols, start="2023-11-01", end="2024-01-31")
data.head()

# Extract only the specified columns
selected_columns = [
    ('Adj Close', 'AAPL'),
    ('Adj Close', 'AMZN'),
    ('Adj Close', 'DIS'),
    ('Adj Close', 'HSY'),
    ('Adj Close', 'JNJ'),
    ('Adj Close', 'META'),
    ('Adj Close', 'MSFT'),
    ('Adj Close', 'T'),
    ('Adj Close', 'UPS'),
    ('Adj Close', 'XOM')
]
selected_data = data[selected_columns]
selected_data.head()


[*********************100%***********************]  10 of 10 completed


Price                       Adj Close                                     \
Ticker                           AAPL        AMZN        DIS         HSY   
Date                                                                       
2023-11-01 00:00:00+00:00  172.894104  137.000000  80.435555  181.392838   
2023-11-02 00:00:00+00:00  176.471848  138.070007  82.638184  183.034409   
2023-11-03 00:00:00+00:00  175.557510  138.600006  84.404251  181.528030   
2023-11-06 00:00:00+00:00  178.121536  139.740005  83.362465  181.209366   
2023-11-07 00:00:00+00:00  180.695557  142.710007  83.928001  181.045197   

Price                                                                     \
Ticker                            JNJ        META        MSFT          T   
Date                                                                       
2023-11-01 00:00:00+00:00  144.129333  310.913849  342.805573  14.671464   
2023-11-02 00:00:00+00:00  145.631805  309.936798  345.034363  14.812716   
2023-11-03 00:00:00+00:00  146.698044  313.655640  349.472046  14.953969   
2023-11-06 00:00:00+00:00  147.046997  314.851990  353.166931  14.831550   
2023-11-07 00:00:00+00:00  146.271545  317.862976  357.129150  14.916302   

Price                                              
Ticker                            UPS         XOM  
Date                                               
2023-11-01 00:00:00+00:00  131.823441  101.229660  
2023-11-02 00:00:00+00:00  133.888504  104.554794  
2023-11-03 00:00:00+00:00  134.708847  103.280312  
2023-11-06 00:00:00+00:00  134.152527  101.450058  
2023-11-07 00:00:00+00:00  133.756485   99.859352

In [4]:
# Save selected data to CSV
selected_data.to_csv('selected_data.csv')

# Verify the file is saved
import os
os.listdir()  # Lists files in the current directory, you should see 'selected_data.csv'


['.config', 'selected_data.csv', 'sample_data']

In [5]:
selected_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 61 entries, 2023-11-01 00:00:00+00:00 to 2024-01-30 00:00:00+00:00
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (Adj Close, AAPL)  61 non-null     float64
 1   (Adj Close, AMZN)  61 non-null     float64
 2   (Adj Close, DIS)   61 non-null     float64
 3   (Adj Close, HSY)   61 non-null     float64
 4   (Adj Close, JNJ)   61 non-null     float64
 5   (Adj Close, META)  61 non-null     float64
 6   (Adj Close, MSFT)  61 non-null     float64
 7   (Adj Close, T)     61 non-null     float64
 8   (Adj Close, UPS)   61 non-null     float64
 9   (Adj Close, XOM)   61 non-null     float64
dtypes: float64(10)
memory usage: 5.2 KB


In [6]:
import plotly.graph_objects as go

fig = go.Figure()

for c in selected_data.columns.values:
    column_name = str(c)  # Convert tuple to string
    fig.add_trace(go.Scatter(x=selected_data.index, y=selected_data[c], mode='lines', name=column_name))

fig.update_layout(
    title='Stock Prices',
    xaxis_title='Date',
    yaxis_title='Price in $',
    legend=dict(x=1, y=1, traceorder='normal', font=dict(family='sans-serif', size=12)),
    margin=dict(l=30, r=20, t=40, b=20)
)

fig.show()


In [7]:
returns = selected_data.pct_change()

fig = go.Figure()

for c in returns.columns.values:
    fig.add_trace(go.Scatter(x=returns.index, y=returns[c], mode='lines', name=str(c)))

fig.update_layout(
    title='Daily Returns',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    legend=dict(x=1, y=1, traceorder='normal', font=dict(family='sans-serif', size=12)),
    margin=dict(l=30, r=20, t=40, b=20)
)

fig.show()


In [8]:
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) * 63
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(63)
    return std, returns

def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3,num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(10)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

In [9]:
returns = selected_data.pct_change()
mean_returns = returns.mean()
cov_matrix = returns.cov()
num_portfolios = 50000
risk_free_rate = 0.043

In [10]:
print(mean_returns)

Price      Ticker
Adj Close  AAPL      0.001384
           AMZN      0.002558
           DIS       0.003168
           HSY       0.000999
           JNJ       0.001260
           META      0.004249
           MSFT      0.002854
           T         0.002317
           UPS       0.000940
           XOM       0.000104
dtype: float64


In [11]:
cov_matrix

Price            Adj Close                                          \
Ticker                AAPL      AMZN       DIS       HSY       JNJ   
Price     Ticker                                                     
Adj Close AAPL    0.000132  0.000080  0.000021 -0.000008  0.000002   
          AMZN    0.000080  0.000149 -0.000010  0.000009 -0.000005   
          DIS     0.000021 -0.000010  0.000267  0.000011  0.000007   
          HSY    -0.000008  0.000009  0.000011  0.000146  0.000023   
          JNJ     0.000002 -0.000005  0.000007  0.000023  0.000068   
          META    0.000074  0.000102 -0.000006 -0.000014 -0.000019   
          MSFT    0.000063  0.000074 -0.000004 -0.000002 -0.000008   
          T       0.000010 -0.000002  0.000030  0.000020  0.000040   
          UPS     0.000065  0.000035  0.000058 -0.000012  0.000052   
          XOM    -0.000021 -0.000003  0.000039  0.000042  0.000031   

Price                                                                       
Ticker                    META      MSFT         T       UPS           XOM  
Price     Ticker                                                            
Adj Close AAPL    7.434992e-05  0.000063  0.000010  0.000065 -2.113989e-05  
          AMZN    1.016191e-04  0.000074 -0.000002  0.000035 -3.276490e-06  
          DIS    -5.525591e-06 -0.000004  0.000030  0.000058  3.918664e-05  
          HSY    -1.448641e-05 -0.000002  0.000020 -0.000012  4.161141e-05  
          JNJ    -1.910820e-05 -0.000008  0.000040  0.000052  3.139742e-05  
          META    1.823581e-04  0.000085 -0.000016  0.000018 -4.673807e-07  
          MSFT    8.543900e-05  0.000098 -0.000007 -0.000005 -3.281870e-05  
          T      -1.640606e-05 -0.000007  0.000162  0.000039  4.294295e-05  
          UPS     1.793088e-05 -0.000005  0.000039  0.000258  2.135311e-05  
          XOM    -4.673807e-07 -0.000033  0.000043  0.000021  1.572632e-04

In [12]:
diagonal_elements = np.diag(cov_matrix)
diagonal_elements

array([1.32280492e-04, 1.48874945e-04, 2.66986247e-04, 1.45703316e-04,
       6.79151337e-05, 1.82358062e-04, 9.78120842e-05, 1.61770677e-04,
       2.58057425e-04, 1.57263238e-04])

In [13]:
def display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, weights = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)

    max_sharpe_idx = np.argmax(results[2])
    sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx],index=selected_data.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T

    min_vol_idx = np.argmin(results[0])
    sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]
    min_vol_allocation = pd.DataFrame(weights[min_vol_idx],index=selected_data.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T

    print("-"*80)
    print("Maximum Sharpe Ratio Portfolio Allocation\n")
    print("Annualised Return:", round(rp, 2))
    print("Annualised Volatility:", round(sdp, 2))
    print("\n")
    print(max_sharpe_allocation)
    print("-"*80)
    print("Minimum Volatility Portfolio Allocation\n")
    print("Annualised Return:", round(rp_min, 2))
    print("Annualised Volatility:", round(sdp_min, 2))
    print("\n")
    print(min_vol_allocation)


    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')
    plt.title('Simulated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)

In [14]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

def display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, weights = random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate)

    max_sharpe_idx = np.argmax(results[2])
    sdp, rp = results[0, max_sharpe_idx], results[1, max_sharpe_idx]
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx], index=selected_data.columns, columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100, 2) for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T

    min_vol_idx = np.argmin(results[0])
    sdp_min, rp_min = results[0, min_vol_idx], results[1, min_vol_idx]
    min_vol_allocation = pd.DataFrame(weights[min_vol_idx], index=selected_data.columns, columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100, 2) for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T

    print("-"*80)
    print("Maximum Sharpe Ratio Portfolio Allocation\n")
    print("Annualised Return:", round(rp, 2))
    print("Annualised Volatility:", round(sdp, 2))
    print("\n")
    print(max_sharpe_allocation)
    print("-"*80)
    print("Minimum Volatility Portfolio Allocation\n")
    print("Annualised Return:", round(rp_min, 2))
    print("Annualised Volatility:", round(sdp_min, 2))
    print("\n")
    print(min_vol_allocation)

    # Scatter plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=results[0,:],
        y=results[1,:],
        mode='markers',
        marker=dict(
            color=results[2,:],
            colorscale='YlGnBu',
            size=10,
            opacity=0.3,
        ),
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=[sdp],
        y=[rp],
        mode='markers',
        marker=dict(
            color='red',
            symbol='star',
            size=14,
        ),
        name='Maximum Sharpe Ratio',
    ))

    fig.add_trace(go.Scatter(
        x=[sdp_min],
        y=[rp_min],
        mode='markers',
        marker=dict(
            color='green',
            symbol='star',
            size=14,
        ),
        name='Minimum Volatility',
    ))

    fig.update_layout(
        title='Simulated Portfolio Optimization based on Efficient Frontier',
        xaxis_title='Annualised Volatility',
        yaxis_title='Annualised Returns',
        legend=dict(
            x=0.02,
            y=0.98,
            bgcolor='rgba(255, 255, 255, 0.5)',
            bordercolor='rgba(255, 255, 255, 0.5)',
            font=dict(size=10),
        ),
        margin=dict(l=30, r=10, t=40, b=10),
    )

    fig.show()


In [15]:
display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)

--------------------------------------------------------------------------------
Maximum Sharpe Ratio Portfolio Allocation

Annualised Return: 0.18
Annualised Volatility: 0.05


Price      Adj Close                                                          
Ticker          AAPL  AMZN    DIS   HSY    JNJ   META   MSFT     T   UPS   XOM
allocation      2.61  4.85  15.52  3.99  20.57  31.72  11.73  6.64  0.54  1.84
--------------------------------------------------------------------------------
Minimum Volatility Portfolio Allocation

Annualised Return: 0.12
Annualised Volatility: 0.04


Price      Adj Close                                                      
Ticker          AAPL  AMZN   DIS    HSY   JNJ  META   MSFT     T  UPS  XOM
allocation      6.04  4.59  8.63  14.87  20.9  8.34  17.03  8.09  2.0  9.5
